In [13]:
%load_ext autoreload
%autoreload 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import spacy
from spacy.lang.en import English
import pickle
plt.style.use('bmh')
import string
from datetime import datetime

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold
%aimport hyperparamfuncs
from hyperparamfuncs import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
x_train = pd.read_pickle('data/review_1pct_bus_svd_x_train.pkl')
x_test = pd.read_pickle('data/review_1pct_bus_svd_x_test.pkl')
y_train = pd.read_pickle('data/sentiment_y_train.pkl')
y_test = pd.read_pickle('data/sentiment_y_test.pkl')

In [3]:
x_train = preprocess_data(x_train)

x_test = preprocess_data(x_test)


/home/gksullan/Documents/Springboard/Capstone_Project_2/hyperparamfuncs.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=1, inplace=True)


In [4]:
hyperparam_table = []

In [5]:
hyper_dict = {'test_size': 0.10, 
              'random_state': 12, 
              'no_of_features': x_train.shape[1],
              'scaling': None,
              'NLP_method': 'SVD',
              'model': LogisticRegression,
              'class_weight': None
             }
clf, hyperparam_table = train_eval(x_train, y_train, x_test, y_test, hyper_dict, hyperparam_table)

/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [6]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,no_of_features,scaling,NLP_method,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances
0,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,None,0.856021,0.858514,2072,703,452,4795,0.892508,0.872135,0.913856,"[[3.104756420826476, -10.560782152568095, 20.9..."


In [7]:
hyper_dict = {'test_size': 0.10, 
              'random_state': 12, 
              'no_of_features': x_train.shape[1],
              'scaling': None,
              'NLP_method': 'SVD',
              'model': LogisticRegression,
              'class_weight': 'balanced'
             }
clf, hyperparam_table = train_eval(x_train, y_train, x_test, y_test, hyper_dict, hyperparam_table)

/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,no_of_features,scaling,NLP_method,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances
0,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,None,0.856021,0.858514,2072,703,452,4795,0.892508,0.872135,0.913856,"[[3.104756420826476, -10.560782152568095, 20.9..."
1,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,balanced,0.846048,0.847240,2365,410,825,4422,0.877468,0.915149,0.842767,"[[3.4589649422649615, -10.537341506629696, 21...."


In [9]:
hyper_dict = {'test_size': 0.10, 
              'random_state': 12, 
              'no_of_features': x_train.shape[1],
              'scaling': None,
              'NLP_method': 'SVD',
              'model': GaussianNB,
              'params': {}
             }
clf, hyperparam_table = train_eval(x_train, y_train, x_test, y_test, hyper_dict, hyperparam_table)

In [10]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,no_of_features,scaling,NLP_method,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances,params
0,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,None,0.856021,0.858514,2072,703,452,4795,0.892508,0.872135,0.913856,"[[3.104756420826476, -10.560782152568095, 20.9...",NaN
1,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,balanced,0.846048,0.847240,2365,410,825,4422,0.877468,0.915149,0.842767,"[[3.4589649422649615, -10.537341506629696, 21....",NaN
2,0.1,12,105,None,SVD,<class 'sklearn.naive_bayes.GaussianNB'>,NaN,0.724757,0.735145,2252,523,1685,3562,0.763395,0.871971,0.678864,None,{}


In [11]:
hyper_dict = {'test_size': 0.10, 
              'random_state': 12, 
              'no_of_features': x_train.shape[1],
              'scaling': None,
              'NLP_method': 'SVD',
              'model': LinearSVC,
              'params': {}
             }
clf, hyperparam_table = train_eval(x_train, y_train, x_test, y_test, hyper_dict, hyperparam_table)

/home/gksullan/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head()

,test_size,random_state,no_of_features,scaling,NLP_method,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances,params
0,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,None,0.856021,0.858514,2072,703,452,4795,0.892508,0.872135,0.913856,"[[3.104756420826476, -10.560782152568095, 20.9...",NaN
1,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,balanced,0.846048,0.847240,2365,410,825,4422,0.877468,0.915149,0.842767,"[[3.4589649422649615, -10.537341506629696, 21....",NaN
2,0.1,12,105,None,SVD,<class 'sklearn.naive_bayes.GaussianNB'>,NaN,0.724757,0.735145,2252,523,1685,3562,0.763395,0.871971,0.678864,None,{}
3,0.1,12,105,None,SVD,<class 'sklearn.svm.classes.LinearSVC'>,NaN,0.836824,0.840854,1728,1047,262,4985,0.883944,0.826426,0.950067,"[[0.18216935032069842, -3.0962481405509332, 5....",{}


In [15]:
hyper_dict = {'test_size': 0.10, 
              'random_state': 12, 
              'no_of_features': x_train.shape[1],
              'scaling': None,
              'NLP_method': 'SVD',
              'model': XGBClassifier,
              'params': {}
             }
clf, hyperparam_table = train_eval(x_train, y_train, x_test, y_test, hyper_dict, hyperparam_table)

In [17]:
df_hyp = pd.DataFrame(hyperparam_table)
df_hyp.head(10)

,test_size,random_state,no_of_features,scaling,NLP_method,model,class_weight,test_score,train_score,tn,fp,fn,tp,f1_score,precision,recall,feature_importances,params
0,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,None,0.856021,0.858514,2072,703,452,4795,0.892508,0.872135,0.913856,"[[3.104756420826476, -10.560782152568095, 20.9...",NaN
1,0.1,12,105,None,SVD,<class 'sklearn.linear_model.logistic.Logistic...,balanced,0.846048,0.847240,2365,410,825,4422,0.877468,0.915149,0.842767,"[[3.4589649422649615, -10.537341506629696, 21....",NaN
2,0.1,12,105,None,SVD,<class 'sklearn.naive_bayes.GaussianNB'>,NaN,0.724757,0.735145,2252,523,1685,3562,0.763395,0.871971,0.678864,None,{}
3,0.1,12,105,None,SVD,<class 'sklearn.svm.classes.LinearSVC'>,NaN,0.836824,0.840854,1728,1047,262,4985,0.883944,0.826426,0.950067,"[[0.18216935032069842, -3.0962481405509332, 5....",{}
4,0.1,12,105,None,SVD,<class 'xgboost.sklearn.XGBClassifier'>,NaN,0.851783,0.934804,2077,698,491,4756,0.888889,0.872021,0.906423,"[0.0064113345, 0.073563255, 0.111385025, 0.005...",{}


In [18]:
# save the hyperparameter table to a .pickle file
time = datetime.now().strftime("%Y%m%d_%H:%M:%S")
filename = 'data/hyperparameter_tables/svd_hyperparameter_table'+time+'.pkl'
df_hyp.to_pickle(filename)